# Clustering Crypto

In [138]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [139]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [140]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [141]:
# Alternatively, use the provided csv file:
file_path = Path('../Resources/crypto_data.csv')

# Create a DataFrame
crypto_df = pd.read_csv(file_path)
crypto_df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


### Data Preprocessing

In [142]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_data =  crypto_df.drop(columns=['Unnamed: 0'])

In [143]:
# Keep only cryptocurrencies that are trading
crypto_data = crypto_data.loc[crypto_data['IsTrading'] == True]

In [144]:
# Keep only cryptocurrencies with a working algorithm
crypto_data = crypto_data.loc[crypto_data['Algorithm'] != 'N/A']

In [145]:
# Remove the "IsTrading" column
crypto_data = crypto_data.drop(columns='IsTrading')

In [146]:
# Remove rows with at least 1 null value
crypto_data = crypto_data.dropna()

In [147]:
# Remove rows with cryptocurrencies having no coins mined
crypto_data = crypto_data[crypto_data['TotalCoinsMined']!=0]

In [148]:
# Drop rows where there are 'N/A' text values
crypto_data = crypto_data.loc[crypto_data['TotalCoinSupply'] != 'N/A']
crypto_data = crypto_data.loc[crypto_data['ProofType'] != 'N/A']

In [149]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
all_crypto_data = crypto_data

In [150]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_data = crypto_data.drop(columns='CoinName')
crypto_data.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
9,Scrypt,PoW,6.303924e+07,84000000
10,X11,PoW/PoS,9.031294e+06,22000000
11,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethash,PoW,1.133597e+08,210000000
13,Equihash,PoW,7.383056e+06,21000000


In [151]:
# Create dummy variables for text features
crypto_dummy = pd.get_dummies(data=crypto_data, columns=['Algorithm', 'ProofType'])

In [152]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_dummy)
crypto_scaled

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [153]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [154]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_data.index
)
pca_df.head()

,PC 1,PC 2,PC 3
0,-0.343822,1.015453,-0.585893
2,-0.327150,1.015500,-0.586407
5,2.302348,1.625255,-0.637253
7,-0.139098,-1.276245,0.196302
8,-0.147640,-1.984634,0.407843


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [155]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Visualization", xlabel="Number of Clusters", ylabel="Inertia")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [156]:
# Initialize the K-Means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pca_df["class"] = model.labels_


clustered_df = pd.concat([crypto_data, pca_df, all_crypto_data], axis=1)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.343822,1.015453,-0.585893,1,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.327150,1.015500,-0.586407,1,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359,2.302348,1.625255,-0.637253,1,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000,-0.139098,-1.276245,0.196302,0,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0,-0.147640,-1.984634,0.407843,0,Ethereum,Ethash,PoW,1.076842e+08,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [166]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

#### Table of Tradable Cryptocurrencies

In [158]:
# Table with tradable cryptos


In [159]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [160]:
# Scale data to create the scatter plot


In [161]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
